In [25]:
import math

import numpy as np
import optuna
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import xgboost as xgb


import mlflow
import pickle

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5003")

In [4]:
def get_or_create_experiment(experiment_name):

    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)

In [5]:
experiment_id = get_or_create_experiment("Scooter Rental Users Prediction")

In [6]:
experiment_id

'706065093627561680'

In [7]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [8]:
import pickle 

with open('preprocess.pkl', 'rb') as f:
    pre = pickle.load(f)

In [9]:
train_x = train.drop('log_total_users', axis=1)
train_y = train.log_total_users
valid_x = test.drop('log_total_users', axis=1)
valid_y = test.log_total_users

In [10]:
train_x_tf = pre.transform(train_x)
valid_x_tf = pre.transform(valid_x)

In [11]:
dtrain = xgb.DMatrix(train_x_tf, label=train_y)
dvalid = xgb.DMatrix(valid_x_tf, label=valid_y)

In [12]:
optuna.logging.set_verbosity(optuna.logging.ERROR)


def champion_callback(study, frozen_trial):

    winner = study.user_attrs.get("winner", None)

    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

In [13]:
def objective(trial):
    with mlflow.start_run(nested=True):
        # Define hyperparameters
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse", 
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"])
        }

        if params["booster"] == "gbtree" or params["booster"] == "dart":
            params["max_depth"] = trial.suggest_int("max_depth", 1, 9)
            params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            params["max_features"] = trial.suggest_categorical("max_features", ["sqrt", "log2"])
            params["max_delta_step"] = trial.suggest_int("max_delta_step", 1, 10, log=True)


        # Train XGBoost model
        bst = xgb.train(params, dtrain)
        preds = bst.predict(dvalid)
        error = mean_squared_error(valid_y, preds)
        perror = mean_absolute_percentage_error(valid_y, preds)

        # Log to MLflow
        mlflow.log_params(params)
        mlflow.log_metric("mse", error)
        mlflow.log_metric("rmse", math.sqrt(error))
        mlflow.log_metric("mape", perror)

    return error

In [14]:
run_name = "first_attempt"

In [15]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    
    study = optuna.create_study(direction="minimize")

    
    study.optimize(objective, n_trials=500, callbacks=[champion_callback])

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_mse", study.best_value)
    mlflow.log_metric("best_rmse", math.sqrt(study.best_value))

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "Scooter Rental Users Project",
            "optimizer_engine": "optuna",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    model = xgb.train(study.best_params, dtrain)


    artifact_path = "model"

    mlflow.xgboost.log_model(
        xgb_model=model,
        artifact_path=artifact_path,
        input_example=train_x.iloc[[0]],
        model_format="ubj",
        metadata={"model_data_version": 1},
    )

    # Get the logged model uri so that we can load it from the artifact store
    model_uri = mlflow.get_artifact_uri(artifact_path)

/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, U

Initial trial 0 achieved value: 1.7476107468249351
Trial 1 achieved value: 1.0354140015391615 with  68.7838% improvement
Trial 4 achieved value: 1.002976647079888 with  3.2341% improvement


/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, U

Trial 10 achieved value: 0.9997462324017763 with  0.3231% improvement


/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, U

...

/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, U

Trial 320 achieved value: 0.1674073880116945 with  0.0948% improvement
Trial 321 achieved value: 0.1672473804773902 with  0.0957% improvement


/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:54:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, U

Trial 451 achieved value: 0.167187287595395 with  0.0359% improvement


/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:55:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:55:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:55:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [15:55:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, U

In [16]:
study.best_params

{'booster': 'gbtree',
 'lambda': 0.40654562199193767,
 'alpha': 0.10795597808162469,
 'max_features': 'log2',
 'max_depth': 9,
 'eta': 0.4199194394913785,
 'gamma': 0.005931158869650323,
 'max_delta_step': 8}

In [17]:
study.best_value

0.167187287595395

#### Fit the model with hyperparameters that minimize the cost functions the most.

In [42]:
xg = xgb.XGBRegressor(booster='gbtree', alpha=0.10796, max_features='log2', max_depth=9, eta=0.42, gamma=0.0059, max_delta_step=8)

In [43]:
xg_model = xg.fit(train_x_tf, train_y)

/Users/dj/anaconda3/envs/mlops_et/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [22:36:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)


In [48]:
with open('xg.pkl', 'wb') as f:
    pickle.dump(xg_model, f)

In [45]:
ypred = xg_model.predict(valid_x_tf)

In [46]:
mean_squared_error(ypred, valid_y)

0.1496853443961339

In [47]:
mean_absolute_percentage_error(ypred, valid_y)

0.04222149758831417